# **Data Builder**

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
data = pd.read_csv("../data/processed_student_data.csv")
data

,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,-0.875498,1.027889,-0.612776,-0.643921,-2.761901,1.242077,1.466302,-1.629761,-1.639229,-1.065328,...,0.068788,-0.195099,0.732511,-0.542374,-0.999995,-0.381387,0.252155,-2.083727,-1.597738,-1.382871
1,-0.875498,0.221035,-0.612776,-0.643921,0.362069,-1.426089,-1.262431,-1.629761,-0.527237,-1.065328,...,1.140653,-0.195099,-0.135527,-0.542374,-0.999995,-0.381387,-0.070060,-2.083727,-1.902291,-1.382871
2,-0.875498,-1.392674,-0.612776,1.552986,0.362069,-1.426089,-1.262431,-1.629761,-0.527237,-0.252314,...,0.068788,-0.195099,-1.003566,0.554987,0.557044,-0.381387,0.896584,-1.413029,-0.988631,-0.347391
3,-0.875498,-1.392674,-0.612776,-0.643921,0.362069,1.242077,-0.352853,-0.826043,0.584754,0.560699,...,-1.003076,-1.165019,-1.003566,-0.542374,-0.999995,1.023086,-0.392275,1.269766,0.838688,0.946958
4,-0.875498,-0.585820,-0.612776,-0.643921,0.362069,0.352689,0.556724,-0.022325,-0.527237,0.560699,...,0.068788,-0.195099,-1.003566,-0.542374,-0.221475,1.023086,-0.070060,-1.748378,-0.379525,-0.347391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,-0.875498,1.834744,1.631918,-0.643921,0.362069,-0.536700,0.556724,0.781392,-0.527237,-1.065328,...,1.140653,0.774821,-1.003566,-0.542374,-0.221475,1.023086,-0.070060,-0.406981,-0.074971,-0.347391
1040,-0.875498,1.027889,-0.612776,1.552986,0.362069,0.352689,-1.262431,1.585110,0.584754,-1.065328,...,0.068788,-0.195099,0.732511,-0.542374,-0.999995,-1.785861,-0.070060,1.269766,1.143242,1.205828
1041,-0.875498,1.027889,-0.612776,-0.643921,0.362069,-1.426089,-1.262431,-0.022325,-0.527237,-1.065328,...,-3.146806,-2.134939,-1.871604,-0.542374,-0.999995,1.023086,0.252155,-0.071631,0.229582,-0.606261
1042,1.142207,0.221035,-0.612776,1.552986,0.362069,0.352689,-1.262431,0.781392,0.584754,-1.065328,...,-2.074941,0.774821,1.600550,1.652347,1.335564,-1.083624,0.252155,-0.406981,-0.379525,-0.347391


In [3]:
data.describe()

,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,...,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03,1.044000e+03
mean,3.402982e-17,2.177909e-16,-9.528351e-17,1.701491e-18,-1.114477e-16,-2.722386e-17,1.633432e-16,1.088954e-16,1.259104e-16,1.361193e-17,...,-9.868649e-17,-1.157014e-16,1.293133e-16,3.402982e-18,1.497312e-16,-3.062684e-17,6.805965e-18,-2.518207e-16,1.361193e-16,-2.654326e-16
std,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,...,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00,1.000479e+00
min,-8.754984e-01,-1.392674e+00,-6.127757e-01,-6.439209e-01,-2.761901e+00,-2.315478e+00,-2.172009e+00,-1.629761e+00,-1.639229e+00,-1.065328e+00,...,-3.146806e+00,-2.134939e+00,-1.871604e+00,-5.423736e-01,-9.999950e-01,-1.785861e+00,-7.144895e-01,-3.760474e+00,-3.425058e+00,-2.936090e+00
25%,-8.754984e-01,-5.858197e-01,-6.127757e-01,-6.439209e-01,3.620694e-01,-5.367001e-01,-1.262431e+00,-8.260430e-01,-5.272373e-01,-1.065328e+00,...,6.878825e-02,-1.950989e-01,-1.003566e+00,-5.423736e-01,-9.999950e-01,-3.813873e-01,-7.144895e-01,-7.423299e-01,-6.840781e-01,-3.473915e-01
50%,-8.754984e-01,2.210349e-01,-6.127757e-01,-6.439209e-01,3.620694e-01,3.526887e-01,-3.528535e-01,-2.232549e-02,-5.272373e-01,-2.523144e-01,...,6.878825e-02,-1.950989e-01,-1.355271e-01,-5.423736e-01,-2.214754e-01,3.208496e-01,-3.922748e-01,-7.163114e-02,-7.497146e-02,-8.852160e-02
75%,1.142207e+00,1.027889e+00,1.631918e+00,1.552986e+00,3.620694e-01,1.242077e+00,5.567243e-01,7.813920e-01,5.847541e-01,5.606987e-01,...,1.140653e+00,7.748213e-01,7.325115e-01,5.549869e-01,5.570442e-01,1.023086e+00,2.521546e-01,5.990676e-01,5.341352e-01,6.880881e-01
max,1.142207e+00,4.255308e+00,1.631918e+00,1.552986e+00,3.620694e-01,1.242077e+00,1.466302e+00,1.585110e+00,2.808737e+00,1.373712e+00,...,1.140653e+00,1.744741e+00,1.600550e+00,3.847069e+00,2.114083e+00,1.023086e+00,1.136856e+01,2.611164e+00,2.361455e+00,2.241307e+00


In [ ]:
# Custom Dataset class
class StudentsDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values  # Assuming the last column is the label
        self.labels = self.data.iloc[:, -1].values
        self.transform = transform

    def __len__(self):
        """Return the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """Retrieve a sample and its label."""
        sample = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)  # Assuming classification

        if self.transform:
            sample = self.transform(sample)

        return sample, label


In [ ]:
# Instantiate the dataset
dataset = StudentsDataset(csv_file='../data/processed_student_data.csv')

# Create a DataLoader to load data in batches
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
